

# <span style="color:#336699">Visualising Z-Scores</span>
<hr style="border:2px solid #0077b9;">
<div style="text-align: left;">
    <a href="https://nbviewer.org/github/swisstopo/topo-satromo/blob/dev-20241209/codegallery/jupyter/Python/stac/stac-introduction.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>
   
<br/>

<b>Abstract.</b> This Jupyter Notebook visualises z-scores and explains what they tell you.


<br/>


In [ ]:
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, timedelta

## Z-score
Z-scores describe how far from the mean a certain value is and thus indicate how "normal" or "rare" it is.


In [ ]:
# Create x values from -4 to 4 standard deviations
x = np.linspace(-4, 4, 1000)

# Calculate the standard normal distribution (mean=0, std=1)
y = stats.norm.pdf(x, 0, 1)

# Create the plot
plt.figure(figsize=(10, 5))

# Define the color gradient
colors = ['#7d6608', '#d4ac0d', '#f7dc6f', '#f5f5f5', '#7dcea0', '#229954', '#145a32']
# positions = [-3, -2, -1, 0, 1, 2, 3]  # Corresponding x positions for color transitions

# Create custom colormap
cmap = LinearSegmentedColormap.from_list('custom', list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the gradient fill
for i in range(len(x)-1):
    # Normalize x value to 0-1 range for colormap (mapping -4 to 4 range to 0-1)
    norm_x = (x[i] + 4) / 8  # Convert -4 to 4 range to 0 to 1 range
    color = cmap(norm_x)
    
    # Fill small segment under curve
    plt.fill_between([x[i], x[i+1]], [0, 0], [y[i], y[i+1]], color=color, alpha=0.7)


plt.plot(x, y, 'k-', linewidth=2, label='Standard Normal Distribution')

# Set axis labels
plt.xlabel('Z-Score', fontsize=14)
plt.ylabel('Probability Density', fontsize=14)

# Remove y-axis tick labels
plt.yticks([])

# Set x-axis ticks at standard deviation intervals
plt.xticks(np.arange(-4, 5), ['-4', '-3', '-2', '-1', '0', '1', '2', '3', '4'], fontsize=14)

# Add vertical lines from x-axis to curve at key z-scores
for i in range(-3, 4):
    y_at_i = stats.norm.pdf(i, 0, 1)  # Get y-value on curve at x=i
    plt.plot([i, i], [0, y_at_i], color='gray', linestyle='-', alpha=0.7, linewidth=1)

# Set axis limits
plt.xlim(-4, 4)
plt.ylim(-0.29, 0.42)  # Extended bottom to accommodate arrows

# Create custom arrows for axes
ax = plt.gca()
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Hide the part of y-axis below zero
ax.spines['left'].set_bounds(0, 0.42)  # Only show y-axis from 0 to top
ax.yaxis.set_label_coords(0.5, 0.7)  # Position y-label in middle of positive range

# Add horizontal double-headed arrows with percentages
arrow_y1 = -0.15
arrow_y2 = -0.22
arrow_y3 = -0.29

# 68% arrow (±1σ)
ax.annotate('', xy=(1, arrow_y1), xytext=(-1, arrow_y1),
            arrowprops=dict(arrowstyle='<->', color='black', lw=1))
ax.text(0, arrow_y1 + 0.01, '68%', ha='center', va='bottom', fontsize=14)

# 95% arrow (±2σ)
ax.annotate('', xy=(2, arrow_y2), xytext=(-2, arrow_y2),
            arrowprops=dict(arrowstyle='<->', color='black', lw=1))
ax.text(0, arrow_y2 + 0.01, '95%', ha='center', va='bottom', fontsize=14)

# 99.7% arrow (±3σ)
ax.annotate('', xy=(3, arrow_y3), xytext=(-3, arrow_y3),
            arrowprops=dict(arrowstyle='<->', color='black', lw=1))
ax.text(0, arrow_y3 + 0.01, '99.7%', ha='center', va='bottom', fontsize=14)

# Adjust layout and show
plt.tight_layout()
plt.show()

# Optional: Add some statistical information
print("Key properties of the standard normal distribution:")
print(f"Mean (μ): 0")
print(f"Standard deviation (σ): 1")
print(f"68% of data falls within ±1σ: [{-1:.1f}, {1:.1f}]")
print(f"95% of data falls within ±2σ: [{-2:.1f}, {2:.1f}]")
print(f"99.7% of data falls within ±3σ: [{-3:.1f}, {3:.1f}]")

## Real values from the GEE

In [ ]:
import ee
import json
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar
from pydrive.auth import GoogleAuth
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
# GEE credentials
config = r'D:\Users\joan-sturm-w1\Documents\GitHub\topo-satromo\secrets\geetest-credentials-int.secret'

Initialize Google Earth Engine (GEE)

In [ ]:
def initialize_gee_and_drive():
    """
    Initializes and authenticates Google Earth Engine (GEE) and Google Drive APIs using service account credentials.
    
    This function:
    1. Sets up authentication scopes for Google Drive
    2. Reads service account key file from the config path
    3. Authenticates with Google Drive using the service account credentials
    4. Initializes Google Earth Engine with the same credentials
    5. Performs a test query to verify GEE initialization was successful
    
    Requirements:
        - PyDrive (for Google Drive authentication)
        - Earth Engine Python API (ee)
        - oauth2client (for ServiceAccountCredentials)
        - Valid service account key file path in config variable
    
    Returns:
        None. Prints confirmation message if initialization was successful or failure message otherwise.
    
    Raises:
        FileNotFoundError: If the service account key file cannot be found
        JSONDecodeError: If the service account key file is not valid JSON
        EEException: If Earth Engine initialization fails
    """
    # Set scopes for Google Drive
    scopes = ["https://www.googleapis.com/auth/drive"]

    # Initialize GEE and authenticate using the service account key file

    # Read the service account key file
    with open(config, "r") as f:
        data = json.load(f)

    # Authenticate with Google using the service account key file
    gauth = GoogleAuth()
    gauth.service_account_file = config
    gauth.service_account_email = data["client_email"]
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(
        gauth.service_account_file, scopes=scopes
    )

    # Initialize Google Earth Engine
    credentials = ee.ServiceAccountCredentials(
        gauth.service_account_email, gauth.service_account_file
    )
    ee.Initialize(credentials)

    # Test if GEE initialization is successful
    image = ee.Image("NASA/NASADEM_HGT/001")
    title = image.get("title").getInfo()

    if title == "NASADEM: NASA NASADEM Digital Elevation 30m":
        print("GEE initialization successful")
    else:
        print("GEE initialization FAILED")

# Authenticate with GEE and GDRIVE
initialize_gee_and_drive()

GEE initialization successful


<hr style="border:1px solid #0077b9;">

### Defining collections, time and other variables:

In [ ]:
# Load the warn regions feature and VHI image collections
regions = ee.FeatureCollection('projects/satromo-prod/assets/res/warnregionen_vhi_2056')
ndvi_ref_col = ee.ImageCollection('projects/satromo-prod/assets/col/1991-2020_NDVI_SWISS')
s2_col = ee.ImageCollection('projects/satromo-prod/assets/col/S2_SR_HARMONIZED_SWISS')

# Set the time period (two months ideally to match swissEO NDVIz)
startDate = '2023-08-01'
endDate = '2023-10-01' # first day outside of desired time period
# Get day of the year for both
ee_date = ee.Date(startDate)
start_doy = int(ee_date.format('D').getInfo())
ee_date = ee.Date(endDate)
end_doy = int(ee_date.format('D').getInfo())
# print(start_doy, end_doy)

In [ ]:
# Defining the region of interest (ROI)

# AOI -> chose specific warn region or ...
# region_nr = 34 #[31-68]  
# region_feature = regions.filter(ee.Filter.eq('REGION_NR', region_nr)).first()
# # Get the region geometry
# aoi = region_feature.geometry()

# AOI -> ... a region by its coordinates
# aoi = ee.Geometry.Rectangle([8.06, 47.14, 8.72, 47.18]) # Raten ZG/SZ
# aoi = ee.Geometry.Rectangle([6.40, 46.47, 6.81, 46.61]) # Lausanne VD
aoi = ee.Geometry.Rectangle([7.37, 46.95, 7.44, 46.97]) # Bremgartenwald, Bern

# Vegetation type: FOREST or ...
vegmask = ee.Image('projects/satromo-prod/assets/res/ch_bafu_lebensraumkarte_mask_forest_epsg32632')
# ... all vegetation
# vegmask = ee.Image('projects/satromo-prod/assets/res/ch_bafu_lebensraumkarte_mask_vegetation_epsg32632')

# Combining aoi and vegetation mask
roi = vegmask.clip(aoi)

# Extract forest-only geometry 
try:
    # Convert the forest mask to vector geometry for sampling
    forest_geometry = roi.select('b1').gte(1).reduceToVectors(
        geometry=aoi,
        scale=30,
        maxPixels=1e9,
        geometryType='polygon'
    ).geometry()
    sampling_geometry = forest_geometry
    print("Using forest-only geometry for sampling")
except:
    # Fallback to AOI if vector conversion fails
    sampling_geometry = aoi
    print("Using full AOI geometry for sampling (fallback)")

Using forest-only geometry for sampling


In [ ]:
# If roi is an image mask
ndvi_ref_col_masked = ndvi_ref_col.map(lambda img: img.updateMask(roi))
s2_col_masked = s2_col.map(lambda img: img.updateMask(roi))

# If aoi is a geometry
# ndvi_ref_col_masked = ndvi_ref_col.map(lambda img: img.clip(aoi))
# s2_col_masked = s2_col.map(lambda img: img.clip(aoi))

# Scale the reference values appropriately
def scale_reference(image):
    """
    Scales the reference image to 0-1 range.
    
    Args:
        image (ee.Image): The input reference image to scale.
        
    Returns:
        ee.Image: The scaled reference image.
    """
    # Get scale and offset from image properties
    scale = ee.Number(image.get('scale'))
    offset = ee.Number(image.get('offset'))
    
    # Scale the 'median' band
    scaled_band = image.select('median').subtract(offset).divide(scale)
    
    return scaled_band.copyProperties(image, image.propertyNames())

# Scale the NDVI reference collection
ndvi_ref_col_scaled = ndvi_ref_col_masked.map(scale_reference)


In [ ]:
def extract_to_dataframe_efficient(image_collection, roi_geometry):
    """
    Efficient extraction using reduceRegions
    Each image becomes a column, each valid pixel becomes a row
    """
    
    def extract_image_values(image):
        # Reduce the image to get pixel values within ROI
        values = image.select(['medain']).reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi_geometry,
            scale=30,  # Adjust your scale
            maxPixels=1e9
        )
        
        # Get DOY or image identifier
        doy = image.get('doy')
        
        return ee.Feature(None, {
            'doy': doy,
            'values': values.get('median')
        })
    
    # Map over the collection
    value_collection = image_collection.map(extract_image_values)
    
    # Get the results
    results = value_collection.getInfo()['features']
    
    # Build DataFrame
    data_dict = {}
    for feature in results:
        doy = feature['properties']['doy']
        values = feature['properties']['values']
        data_dict[f'doy_{doy}'] = values
    
    # Handle different lengths
    max_length = max(len(v) for v in data_dict.values() if v is not None)
    for key in data_dict:
        if data_dict[key] is None:
            data_dict[key] = []
        data_dict[key].extend([None] * (max_length - len(data_dict[key])))
    
    df = pd.DataFrame(data_dict)
    return df

# Extract to DataFrame
df = extract_to_dataframe_efficient(ndvi_ref_col_scaled, aoi)

# Display info
print(f"DataFrame shape: {df.shape}")
print(df.head())

In [ ]:
# Calculate the median NDVI for swissEO S2-SR image collection
def loadNdviCurrentData(image):
    """
    Loads the current NDVI data from Sentinel-2 imagery.

    Args:
        image (ee.ImageCollection): Sentinel-2 image collection.

    Returns:
        ee.Image: Combined image with median NDVI and pixel count bands
    """
    # Apply the cloud and terrain shadow mask within the S2 image collection
    def applyMasks(image):
        image = image.updateMask(image.select('terrainShadowMask').lt(65))
        image = image.updateMask(image.select('cloudAndCloudShadowMask').eq(0))
        image = image.updateMask(image.select('ndsi').lt(0.43))
        return image
    S2_col_masked = image.map(applyMasks)

    # Calculate NDVI for each image
    def calculate_ndvi(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
        return ndvi
    
    ndvi_col = S2_col_masked.map(calculate_ndvi)
    
    # Calculate median NDVI
    ndvi_median = ndvi_col.median().rename('median')
    
    return ndvi_median

# Calculate NDVI for the current S2 collection
ndvi_s2_col_masked = loadNdviCurrentData(s2_col_masked)

In [ ]:
# Calculate and add z-score band to image collection
def add_zscore_band(image):
    """
    Add a z-score band to each image.
    
    Args:
        collection: ee.ImageCollection - The NDVI reference collection with statistics
    
    Returns:
        ee.ImageCollection - Collection with z-score band added to each image
    """    
    # Calculate mean and std of the median band across the entire collection 
    # within the forest stand area
    ndvi = image.select('median')
    
    # Calculate mean and std of the ndvi values
    mean = ndvi.reduceRegion(ee.Reducer.mean())
    std = ndvi.reduceRegion(ee.Reducer.stdDev())
    
    # Calculate z-score: (median - mean) / std
    zscore = ndvi.subtract(ee.Number(mean)).divide(ee.Number(std))
    zscore = zscore.rename('zscore')
    
    return image.addBands(zscore)

# Map the z-score calculation over the entire collection
ndvi_ref_col_z = ndvi_ref_col_masked.map(add_zscore_band)
ndvi_s2_col_z = add_zscore_band(ndvi_s2_col_masked)

In [ ]:
# Sample a subset of pixels with stratified sampling
def extract_ndvi_data_sampled(image_collection, max_pixels_per_image=5000, scale=100):
    """
    Extract NDVI data using regular sampling with larger scale to avoid pixel limits
    """
    def sample_image(image):
        # Add DOY to the image
        doy = ee.Number(image.get('system:index'))
        
        # Use regular sample with larger scale to reduce pixel count
        sample = image.select(['median', 'z_score']).sample(
            region=image.geometry(),
            scale=scale,  # Larger scale = fewer pixels to process
            numPixels=max_pixels_per_image,
            geometries=False
        )
        
        # Add DOY to each sample
        def add_doy(feature):
            return feature.set('doy', doy)
        
        return sample.map(add_doy)
    
    # Map over collection and flatten
    samples = image_collection.map(sample_image).flatten()
    
    # Get sample data
    sample_data = samples.getInfo()['features']
    
    data = []
    for feature in sample_data:
        props = feature['properties']
        if props['median'] is not None and props['z_score'] is not None:
            data.append({
                'doy': int(props['doy']),
                'median': props['median'],
                'z_score': props['z_score']
            })
    
    return pd.DataFrame(data)

df = extract_ndvi_data_sampled(ndvi_ref_col_z, max_pixels_per_image=5000, scale=100)
print(f"Extracted {len(df)} data points")

In [ ]:
# Create the plot
def create_ndvi_plot(df):
    """
    Create the NDVI plot with z-score coloring
    """
    # Define the color mapping for z-scores
    z_colors = {
        -5: '#7d6608',
        -3: '#d4ac0d', 
        -1: '#f7dc6f',
        0: '#f5f5f5',
        1: '#7dcea0',
        3: '#229954',
        5: '#145a32'
    }
    
    # Create a custom colormap
    z_values = sorted(z_colors.keys())
    colors = [z_colors[z] for z in z_values]
    n_bins = 100
    cmap = mcolors.LinearSegmentedColormap.from_list('z_score', colors, N=n_bins)
    
    # Convert DOY to dates for better x-axis labels
    def doy_to_date(doy, year=2020):  # Using 2020 as a reference year (leap year)
        return datetime(year, 1, 1) + timedelta(days=doy - 1)
    
    df['date'] = df['doy'].apply(doy_to_date)
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Create scatter plot
    scatter = ax.scatter(df['date'], df['median'], 
                        c=df['z_score'], 
                        cmap=cmap, 
                        vmin=-5, vmax=5,
                        alpha=0.6, 
                        s=20)
    
    # Customize the plot
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('NDVI Median Values', fontsize=12)
    ax.set_title('NDVI Time Series with Z-Score Coloring', fontsize=14, fontweight='bold')
    
    # Format x-axis to show months
    import matplotlib.dates as mdates
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_minor_locator(mdates.WeekdayLocator())
    
    # Rotate x-axis labels for better readability
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Z-Score', fontsize=12)
    
    # Add grid for better readability
    ax.grid(True, alpha=0.3)
    
    # Tight layout to prevent label cutoff
    plt.tight_layout()
    
    return fig, ax

# Create and display the plot
fig, ax = create_ndvi_plot(df)
plt.show()